In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

from nnx import autograd
from nnx.autograd.activations import ReLU, Softmax
from nnx.autograd.initialisation import xavier_uniform
from nnx.autograd.layers import Conv2D
from nnx.autograd.tensor import Tensor

seed = 3

autograd.rng = np.random.default_rng(seed=seed)

In [ ]:
from nnx.autograd.layers import Layer


class Linear(Layer):
    """Implements an linear transformation."""

    def __init__(
        self,
        in_dim: int,
        out_dim: int,
        *,
        initialiser: callable,
        bias: bool = True,
    ) -> None:
        """C'tor of the Linear layer.

        Args:
            in_dim: count of input neurons.
            out_dim: count of output neurons.
            initialiser: callable to initialise layers.
            bias: whether we want to use the bias term.

        """
        self._in_dim = in_dim
        self._out_dim = out_dim

        weights, bias_ = initialiser(
            in_dim,
            out_dim,
            size=(out_dim, in_dim),
        )

        self._weights = Tensor(weights, requires_grad=True)
        self._bias = Tensor(bias_, requires_grad=True) if bias else None

    def forward(self, inputs: Tensor) -> Tensor:
        """Compute the transformation given the inputs.

        Args:
            inputs: Tensor which needs to be transformed.

        Returns:
            Transformed Tensor.

        """
        outputs: Tensor = self._weights @ inputs

        outputs.prev = {inputs, self._weights}
        if self._bias is not None:
            outputs.prev.add(self._bias)
            outputs += self._bias

        def _backward() -> None:
            pass

        outputs.register_backward(_backward)

        return outputs

In [4]:
inputs = Tensor(autograd.rng.random((1, 32, 32, 3)), requires_grad=True)
layer_1 = Conv2D((3, 3), 3, 64, initialiser=xavier_uniform)
act_1 = ReLU()
layer_2 = Conv2D((3, 3), 64, 128, initialiser=xavier_uniform)
act_2 = ReLU()
layer_3 = Conv2D((3, 3), 128, 3, initialiser=xavier_uniform)
act_3 = Softmax()


outputs = act_3(layer_3(act_2(layer_2(act_1(layer_1(inputs))))))

In [5]:
mock_label = Tensor(np.ones_like(outputs.data))

In [6]:
outputs.backward(np.ones_like(outputs.data))